In [2]:
import os
os.system("git clone https://github.com/soerenetler/squad-question-generation.git")
os.system("git clone https://github.com/soerenetler/spacy_target_answer.git")
os.system("git clone https://github.com/soerenetler/crf_target_answer.git")

from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
!(cd squad-question-generation/; git pull)
!(cd crf_target_answer/; git pull)

Already up to date.
Already up to date.


In [16]:
!cp crf_target_answer/Custom.py Custom.py

In [4]:
!pip install -r crf_target_answer/requirements.txt --quiet

     |████████████████████████████████| 4.9MB 7.3MB/s 
     |████████████████████████████████| 112kB 52.3MB/s 
     |████████████████████████████████| 747kB 47.7MB/s 
ERROR: yellowbrick 0.9.1 has requirement scikit-learn>=0.20, but you'll have scikit-learn 0.19.2 which is incompatible.
ERROR: imbalanced-learn 0.4.3 has requirement scikit-learn>=0.20, but you'll have scikit-learn 0.19.2 which is incompatible.
ERROR: eli5 0.11.0 has requirement scikit-learn>=0.20, but you'll have scikit-learn 0.19.2 which is incompatible.


In [22]:
!pip install scikit-learn==0.19.2

In [2]:
import ast
import itertools
import pickle
from tqdm import tqdm
import spacy
import pandas as pd

from sklearn.metrics import make_scorer, log_loss, roc_auc_score
from sklearn.model_selection import RandomizedSearchCV

import sklearn_crfsuite

from Custom import *
import plotly.express as px
import joblib

In [3]:
model_folder = "/content/gdrive/MyDrive/mt-qg-data/00_models/crf_target_answer/"
dataset = "squad"

In [4]:
gs = joblib.load(model_folder + dataset + '/gs.pkl')

In [5]:
pd.DataFrame(gs.cv_results_).columns

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:122: FutureWarning:

You are accessing a training score ('split0_train_roc_auc'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:122: FutureWarning:

You are accessing a training score ('split1_train_roc_auc'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:122: FutureWarning:

You are accessing a training score ('split2_train_roc_auc'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:122: FutureWarning:

You are accessing a training score ('split3_train_roc_auc'), which will not be available by def

Index(['mean_fit_time', 'std_fit_time', 'mean_score_time', 'std_score_time',
       'param_c1', 'param_c2', 'params', 'split0_test_roc_auc',
       'split1_test_roc_auc', 'split2_test_roc_auc', 'split3_test_roc_auc',
       'split4_test_roc_auc', 'mean_test_roc_auc', 'std_test_roc_auc',
       'rank_test_roc_auc', 'split0_train_roc_auc', 'split1_train_roc_auc',
       'split2_train_roc_auc', 'split3_train_roc_auc', 'split4_train_roc_auc',
       'mean_train_roc_auc', 'std_train_roc_auc', 'split0_test_log_loss',
       'split1_test_log_loss', 'split2_test_log_loss', 'split3_test_log_loss',
       'split4_test_log_loss', 'mean_test_log_loss', 'std_test_log_loss',
       'rank_test_log_loss', 'split0_train_log_loss', 'split1_train_log_loss',
       'split2_train_log_loss', 'split3_train_log_loss',
       'split4_train_log_loss', 'mean_train_log_loss', 'std_train_log_loss'],
      dtype='object')

In [6]:
import plotly.express as px
_x = [s['c1'] for s in gs.cv_results_["params"]]
_y = [s['c2'] for s in gs.cv_results_["params"]]
_log_loss = [s for s in gs.cv_results_["mean_test_log_loss"]]
_roc_auc = [s for s in gs.cv_results_["mean_test_roc_auc"]]

print("Dark blue => {:0.4}, dark red => {:0.4}".format(min(_log_loss), max(_log_loss)))

print('best params:', gs.best_params_)
print('best CV score:', gs.best_score_)
print('model size: {:0.2f}M'.format(gs.best_estimator_.size_ / 1000000))

fig = px.scatter(
                 x=_x,
                 y=_y,
                 color=_log_loss,
                 log_x=True, log_y=True)
fig.show()

Dark blue => 0.3865, dark red => 0.6629
best params: {'c1': 10, 'c2': 0.1}
best CV score: 0.708879051072158
model size: 0.42M


In [7]:
fig = px.scatter(
                 x=_x,
                 y=_y,
                 color=_roc_auc,
                 log_x=True, log_y=True)
fig.show()